# Getting Started with pandas

pandas will be a major tool of interest throughout much of the rest of the course. It
contains data structures and data manipulation tools designed to make data cleaning
and analysis fast and easy in Python. pandas is often used in tandem with numerical
computing tools like NumPy and SciPy, analytical libraries like statsmodels and
scikit-learn, and data visualization libraries like matplotlib. pandas adopts significant
parts of NumPy’s idiomatic style of array-based computing, especially array-based
functions and a preference for data processing without for loops.

While pandas adopts many coding idioms from NumPy, the biggest difference is that
pandas is designed for working with tabular or heterogeneous data. NumPy, by contrast,
is best suited for working with homogeneous numerical array data.

we use the following import convention for pandas:

In [2]:
import pandas as pd

Thus, whenever you see pd. in code, it’s referring to pandas. You may also find it easier
to import Series and DataFrame into the local namespace since they are so frequently
used:

In [3]:
from pandas import Series, DataFrame

In [4]:
import numpy as np
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

## Introduction to pandas Data Structures

To get started with pandas, you will need to get comfortable with its two workhorse
data structures: Series and DataFrame. While they are not a universal solution for
every problem, they provide a solid, easy-to-use basis for most applications

### Series

A Series is a one-dimensional array-like object containing a sequence of values (of
similar types to NumPy types) and an associated array of data labels, called its index.
The simplest Series is formed from only an array of data:

In [5]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

The string representation of a Series displayed interactively shows the index on the
left and the values on the right. Since we did not specify an index for the data, a
default one consisting of the integers 0 through N - 1 (where N is the length of the
data) is created. You can get the array representation and index object of the Series via
its values and index attributes, respectively:

In [6]:
obj.values

array([ 4,  7, -5,  3])

In [7]:
obj.index  # like range(4)

RangeIndex(start=0, stop=4, step=1)

Often it will be desirable to create a Series with an index identifying each data point
with a label:

In [8]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [9]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

Compared with NumPy arrays, you can use labels in the index when selecting single
values or a set of values:

In [10]:
obj2['a']

-5

In [11]:
obj2['d'] = 6

In [12]:
obj2[['c', 'a', 'd']]

c    3
a   -5
d    6
dtype: int64

Here ['c', 'a', 'd'] is interpreted as a list of indices, even though it contains
strings instead of integers.

Using NumPy functions or NumPy-like operations, such as filtering with a boolean
array, scalar multiplication, or applying math functions, will preserve the index-value
link:

In [13]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [14]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [15]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

Should you have data contained in a Python dict, you can create a Series from it by
passing the dict:

In [16]:
sdata = {'Lisboa': 35000, 'Alentejoo': 71000, 'Porto': 16000, 'Coimbra': 5000}
obj3 = pd.Series(sdata)
obj3

Lisboa       35000
Alentejoo    71000
Porto        16000
Coimbra       5000
dtype: int64

When you are only passing a dict, the index in the resulting Series will have the dict’s
keys in sorted order. You can override this by passing the dict keys in the order you
want them to appear in the resulting Series:

In [17]:
states = ['Aveiro', 'Coimbra', 'Lisboa', 'Porto']
obj4 = pd.Series(sdata, index=states)
obj4

Aveiro         NaN
Coimbra     5000.0
Lisboa     35000.0
Porto      16000.0
dtype: float64

Here, three values found in sdata were placed in the appropriate locations, but since
no value for 'Aveiro' was found, it appears as NaN (not a number), which is considered
in pandas to mark missing or NA values. Since 'Alentejoo' was not included in
states, it is excluded from the resulting object.

The
isnull and notnull functions in pandas should be used to detect missing data:

In [18]:
pd.isnull(obj4)

Aveiro      True
Coimbra    False
Lisboa     False
Porto      False
dtype: bool

In [19]:
pd.notnull(obj4)

Aveiro     False
Coimbra     True
Lisboa      True
Porto       True
dtype: bool

Series also has these as instance methods:

In [20]:
obj4.isnull()

Aveiro      True
Coimbra    False
Lisboa     False
Porto      False
dtype: bool

A useful Series feature for many applications is that it automatically aligns by index
label in arithmetic operations:

In [21]:
obj3

Lisboa       35000
Alentejoo    71000
Porto        16000
Coimbra       5000
dtype: int64

In [22]:
obj4.fillna(0)

Aveiro         0.0
Coimbra     5000.0
Lisboa     35000.0
Porto      16000.0
dtype: float64

In [23]:
obj3 + obj4

Alentejoo        NaN
Aveiro           NaN
Coimbra      10000.0
Lisboa       70000.0
Porto        32000.0
dtype: float64

Data alignment features will be addressed in more detail later. If you have experience
with databases, you can think about this as being similar to a join operation.

Both the Series object itself and its index have a name attribute, which integrates with
other key areas of pandas functionality:

In [24]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
Aveiro         NaN
Coimbra     5000.0
Lisboa     35000.0
Porto      16000.0
Name: population, dtype: float64

A Series’s index can be altered in-place by assignment:

In [25]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [26]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### DataFrame

A DataFrame represents a rectangular table of data and contains an ordered collection
of columns, each of which can be a different value type (numeric, string,
boolean, etc.). The DataFrame has both a row and column index; it can be thought of
as a dict of Series all sharing the same index

In [27]:
data = {'state': ['Aveiro', 'Aveiro', 'Aveiro',
                  'Coimbra', 'Coimbra', 'Coimbra'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

In [28]:
frame

,state,year,pop
0,Aveiro,2000,1.5
1,Aveiro,2001,1.7
2,Aveiro,2002,3.6
3,Coimbra,2001,2.4
4,Coimbra,2002,2.9
5,Coimbra,2003,3.2


The resulting DataFrame will have its index assigned automatically as with Series, and
the columns are placed in sorted order:

For large DataFrames, the head method selects only the first five rows:

In [29]:
frame.head()

,state,year,pop
0,Aveiro,2000,1.5
1,Aveiro,2001,1.7
2,Aveiro,2002,3.6
3,Coimbra,2001,2.4
4,Coimbra,2002,2.9


If you specify a sequence of columns, the DataFrame’s columns will be arranged in
that order:

In [30]:
pd.DataFrame(data, columns=['year', 'state'])

,year,state
0,2000,Aveiro
1,2001,Aveiro
2,2002,Aveiro
3,2001,Coimbra
4,2002,Coimbra
5,2003,Coimbra


If you pass a column that isn’t contained in the dict, it will appear with missing values
in the result:

In [31]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'popu', 'debt'],
                      index=['one', 'two', 'three', 'four',
                             'five', 'six'])
frame2

,year,state,popu,debt
one,2000,Aveiro,NaN,NaN
two,2001,Aveiro,NaN,NaN
three,2002,Aveiro,NaN,NaN
four,2001,Coimbra,NaN,NaN
five,2002,Coimbra,NaN,NaN
six,2003,Coimbra,NaN,NaN


In [32]:
frame2.columns

Index(['year', 'state', 'popu', 'debt'], dtype='object')

A column in a DataFrame can be retrieved as a Series either by dict-like notation or
by attribute:

In [33]:
frame2['state']

one       Aveiro
two       Aveiro
three     Aveiro
four     Coimbra
five     Coimbra
six      Coimbra
Name: state, dtype: object

In [34]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

Rows can also be retrieved by position or name with the special loc attribute:

In [35]:
frame2.loc['three']

year       2002
state    Aveiro
popu        NaN
debt        NaN
Name: three, dtype: object

In [36]:
frame2.iloc[2]

year       2002
state    Aveiro
popu        NaN
debt        NaN
Name: three, dtype: object

In [37]:
frame2.loc['one', :]

year       2000
state    Aveiro
popu        NaN
debt        NaN
Name: one, dtype: object

In [38]:
frame2.iloc[:, 1]

one       Aveiro
two       Aveiro
three     Aveiro
four     Coimbra
five     Coimbra
six      Coimbra
Name: state, dtype: object

Columns can be modified by assignment. For example, the empty 'debt' column
could be assigned a scalar value or an array of values:

In [39]:
frame2['debt'] = 16.5
frame2

,year,state,popu,debt
one,2000,Aveiro,NaN,16.5
two,2001,Aveiro,NaN,16.5
three,2002,Aveiro,NaN,16.5
four,2001,Coimbra,NaN,16.5
five,2002,Coimbra,NaN,16.5
six,2003,Coimbra,NaN,16.5


In [40]:
frame2['debt'] = np.arange(6.)
frame2

,year,state,popu,debt
one,2000,Aveiro,NaN,0.0
two,2001,Aveiro,NaN,1.0
three,2002,Aveiro,NaN,2.0
four,2001,Coimbra,NaN,3.0
five,2002,Coimbra,NaN,4.0
six,2003,Coimbra,NaN,5.0


When you are assigning lists or arrays to a column, the value’s length must match the
length of the DataFrame. If you assign a Series, its labels will be realigned exactly to
the DataFrame’s index, inserting missing values in any holes:

In [41]:
val = pd.Series([-1.2, -1.5, -1.7],
                index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,popu,debt
one,2000,Aveiro,NaN,NaN
two,2001,Aveiro,NaN,-1.2
three,2002,Aveiro,NaN,NaN
four,2001,Coimbra,NaN,-1.5
five,2002,Coimbra,NaN,-1.7
six,2003,Coimbra,NaN,NaN


Assigning a column that doesn’t exist will create a new column. The del keyword will
delete columns as with a dict.

As an example of del, I first add a new column of boolean values where the state
column equals 'Ohio':

In [42]:
frame2['eastern'] = frame2.state == 'Aveiro'
frame2

,year,state,popu,debt,eastern
one,2000,Aveiro,NaN,NaN,True
two,2001,Aveiro,NaN,-1.2,True
three,2002,Aveiro,NaN,NaN,True
four,2001,Coimbra,NaN,-1.5,False
five,2002,Coimbra,NaN,-1.7,False
six,2003,Coimbra,NaN,NaN,False


The del method can then be used to remove this column:

In [43]:
del frame2['eastern']
frame2.columns

Index(['year', 'state', 'popu', 'debt'], dtype='object')

Another common form of data is a nested dict of dicts:

In [44]:
pop = {'Aveiro': {2001: 2.4, 2002: 2.9},
       'Coimbra': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

In [45]:
frame3 = pd.DataFrame(pop)
frame3

,Aveiro,Coimbra
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


You can transpose the DataFrame (swap rows and columns) with similar syntax to a
NumPy array:.

In [46]:
frame3.T

,2001,2002,2000
Aveiro,2.4,2.9,NaN
Coimbra,1.7,3.6,1.5


Dicts of Series are treated in much the same way:

In [47]:
pdata = {'Aveiro': frame3['Aveiro'][:-1],
         'Coimbra': frame3['Coimbra'][:2]}
pd.DataFrame(pdata)

,Aveiro,Coimbra
2001,2.4,1.7
2002,2.9,3.6


If a DataFrame’s index and columns have their name attributes set, these will also be
displayed:

In [48]:
frame3.index.name = 'year'; frame3.columns.name = 'state'
frame3

state,Aveiro,Coimbra
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


As with Series, the values attribute returns the data contained in the DataFrame as a
two-dimensional ndarray:

In [49]:
frame3.values

array([[2.4, 1.7],
       [2.9, 3.6],
       [nan, 1.5]])

In [50]:
frame2.values

array([[2000, 'Aveiro', nan, nan],
       [2001, 'Aveiro', nan, -1.2],
       [2002, 'Aveiro', nan, nan],
       [2001, 'Coimbra', nan, -1.5],
       [2002, 'Coimbra', nan, -1.7],
       [2003, 'Coimbra', nan, nan]], dtype=object)

### Index Objects

pandas’s Index objects are responsible for holding the axis labels and other metadata
(like the axis name or names). Any array or other sequence of labels you use when
constructing a Series or DataFrame is internally converted to an Index:

In [51]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

In [52]:
index[1:]

Index(['b', 'c'], dtype='object')

Index objects are immutable and thus can’t be modified by the user:

In [53]:
index[1] = 'd'  # TypeError

TypeError: Index does not support mutable operations

Immutability makes it safer to share Index objects among data structures:

In [54]:
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [55]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [56]:
obj2.index is labels

True

In addition to being array-like, an Index also behaves like a fixed-size set:

In [57]:
frame3

state,Aveiro,Coimbra
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


In [58]:
frame3.columns

Index(['Aveiro', 'Coimbra'], dtype='object', name='state')

In [59]:
'Aveiro' in frame3.columns

True

In [103]:
2003 in frame3.index

False

Unlike Python sets, a pandas Index can contain duplicate labels:

In [60]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

## Essential Functionality

This section will walk you through the fundamental mechanics of interacting with the
data contained in a Series or DataFrame

### Reindexing

An important method on pandas objects is reindex, which means to create a new
object with the data conformed to a new index. Consider an example:

In [61]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6],
                index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

Calling reindex on this Series rearranges the data according to the new index, introducing
missing values if any index values were not already present:

In [62]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

For ordered data like time series, it may be desirable to do some interpolation or filling
of values when reindexing. The method option allows us to do this, using a
method such as ffill, which forward-fills the values:

In [63]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [64]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

With DataFrame, reindex can alter either the (row) index, columns, or both. When
passed only a sequence, it reindexes the rows in the result:

In [65]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Aveiro', 'Porto', 'Lisboa'])
frame

,Aveiro,Porto,Lisboa
a,0,1,2
c,3,4,5
d,6,7,8


In [66]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Aveiro,Porto,Lisboa
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


The columns can be reindexed with the columns keyword:

In [67]:
states = ['Aveiro', 'Coimbra', 'Porto']
frame.reindex(columns=states)

,Aveiro,Coimbra,Porto
a,0,NaN,1
c,3,NaN,4
d,6,NaN,7


### Dropping Entries from an Axis

Dropping one or more entries from an axis is easy if you already have an index array
or list without those entries. As that can require a bit of munging and set logic, the
drop method will return a new object with the indicated value or values deleted from
an axis:

In [68]:
obj = pd.Series(np.arange(5.),
                index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [69]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [70]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

With DataFrame, index values can be deleted from either axis. To illustrate this, we
first create an example DataFrame:

In [71]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Aveiro', 'Porto', 'Lisboa', 'Coimbra'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Aveiro,0,1,2,3
Porto,4,5,6,7
Lisboa,8,9,10,11
Coimbra,12,13,14,15


Calling drop with a sequence of labels will drop values from the row labels (axis 0):

In [72]:
data.drop(['Porto', 'Coimbra'])

,one,two,three,four
Aveiro,0,1,2,3
Lisboa,8,9,10,11


You can drop values from the columns by passing axis=1 or axis='columns':

In [73]:
data.drop('two', axis=1)

,one,three,four
Aveiro,0,2,3
Porto,4,6,7
Lisboa,8,10,11
Coimbra,12,14,15


In [74]:
data.drop(['two', 'four'], axis='columns')

,one,three
Aveiro,0,2
Porto,4,6
Lisboa,8,10
Coimbra,12,14


Many functions, like drop, which modify the size or shape of a Series or DataFrame,
can manipulate an object in-place without returning a new object:

In [75]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

### Indexing, Selection, and Filtering

Series indexing (obj[...]) works analogously to NumPy array indexing, except you
can use the Series’s index values instead of only integers. Here are some examples of
this:

In [76]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [77]:
obj['b']

1.0

In [78]:
obj[1]

1.0

In [79]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [80]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [81]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [82]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

Slicing with labels behaves differently than normal Python slicing in that the endpoint
is inclusive:

In [83]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

Setting using these methods modifies the corresponding section of the Series:

In [84]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

Indexing into a DataFrame is for retrieving one or more columns either with a single
value or sequence:

In [85]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Aveiro', 'Lisboa', 'Coimbra', 'Porto'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Aveiro,0,1,2,3
Lisboa,4,5,6,7
Coimbra,8,9,10,11
Porto,12,13,14,15


In [86]:
data['two']

Aveiro      1
Lisboa      5
Coimbra     9
Porto      13
Name: two, dtype: int64

In [87]:
data[['three', 'one']]

,three,one
Aveiro,2,0
Lisboa,6,4
Coimbra,10,8
Porto,14,12


Indexing like this has a few special cases. First, slicing or selecting data with a boolean
array:

In [88]:
data[:2]

,one,two,three,four
Aveiro,0,1,2,3
Lisboa,4,5,6,7


In [89]:
data['three'] > 5

Aveiro     False
Lisboa      True
Coimbra     True
Porto       True
Name: three, dtype: bool

In [90]:
data[data['three'] > 5]

,one,two,three,four
Lisboa,4,5,6,7
Coimbra,8,9,10,11
Porto,12,13,14,15


The row selection syntax data[:2] is provided as a convenience. Passing a single element
or a list to the [] operator selects columns.

Another use case is in indexing with a boolean DataFrame, such as one produced by a
scalar comparison

In [91]:
data < 5

,one,two,three,four
Aveiro,True,True,True,True
Lisboa,True,False,False,False
Coimbra,False,False,False,False
Porto,False,False,False,False


In [92]:
data[data < 5] = 0

In [93]:
data

,one,two,three,four
Aveiro,0,0,0,0
Lisboa,0,5,6,7
Coimbra,8,9,10,11
Porto,12,13,14,15


#### Selection with loc and iloc

For DataFrame label-indexing on the rows, I introduce the special indexing operators
loc and iloc. They enable you to select a subset of the rows and columns from a
DataFrame with NumPy-like notation using either axis labels (loc) or integers
(iloc).

let’s select a single row and multiple columns by label:

In [94]:
data.loc['Coimbra', ['two', 'three']]

two       9
three    10
Name: Coimbra, dtype: int64

We’ll then perform some similar selections with integers using iloc:

In [95]:
data.iloc[2, [1, 2]]

two       9
three    10
Name: Coimbra, dtype: int64

In [96]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Coimbra, dtype: int64

In [97]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Lisboa,7,0,5
Coimbra,11,8,9


Both indexing functions work with slices in addition to single labels or lists of labels:

In [98]:
data.loc[:'Porto', 'two']

Aveiro      0
Lisboa      5
Coimbra     9
Porto      13
Name: two, dtype: int64

In [99]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Lisboa,0,5,6
Coimbra,8,9,10
Porto,12,13,14


### Integer Indexes

Working with pandas objects indexed by integers is something that often trips up
new users due to some differences with indexing semantics on built-in Python data
structures like lists and tuples. For example, you might not expect the following code
to generate an error:

In [100]:
ser = pd.Series(np.arange(3.))
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [101]:
ser[-1]

KeyError: -1

In this case, pandas could “fall back” on integer indexing, but it’s difficult to do this in
general without introducing subtle bugs. Here we have an index containing 0, 1, 2,
but inferring what the user wants (label-based indexing or position-based) is difficult:

In [104]:
ser = pd.Series(np.arange(3.))

In [105]:
ser

0    0.0
1    1.0
2    2.0
dtype: float64

On the other hand, with a non-integer index, there is no potential for ambiguity:

In [106]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

2.0

To keep things consistent, if you have an axis index containing integers, data selection
will always be label-oriented. For more precise handling, use loc (for labels) or iloc
(for integers):

In [107]:
ser[:1]

0    0.0
dtype: float64

In [108]:
ser.loc[:1]

0    0.0
1    1.0
dtype: float64

In [109]:
ser.iloc[:1]

0    0.0
dtype: float64

### Arithmetic and Data Alignment

An important pandas feature for some applications is the behavior of arithmetic
between objects with different indexes. When you are adding together objects, if any
index pairs are not the same, the respective index in the result will be the union of the
index pairs.

In [110]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=['a', 'c', 'e', 'f', 'g'])

In [111]:
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [112]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

Adding these together yields:

In [113]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

The internal data alignment introduces missing values in the label locations that don’t
overlap. Missing values will then propagate in further arithmetic computations.

In the case of DataFrame, alignment is performed on both the rows and the columns:

In [114]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                   index=['Coimbra', 'Porto', 'Lisboa'])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Aveiro', 'Coimbra', 'Porto', 'Lisboa'])

In [115]:
df1

,b,c,d
Coimbra,0.0,1.0,2.0
Porto,3.0,4.0,5.0
Lisboa,6.0,7.0,8.0


In [116]:
df2

,b,d,e
Aveiro,0.0,1.0,2.0
Coimbra,3.0,4.0,5.0
Porto,6.0,7.0,8.0
Lisboa,9.0,10.0,11.0


Adding these together returns a DataFrame whose index and columns are the unions
of the ones in each DataFrame:

In [117]:
df1 + df2

,b,c,d,e
Aveiro,NaN,NaN,NaN,NaN
Coimbra,3.0,NaN,6.0,NaN
Lisboa,15.0,NaN,18.0,NaN
Porto,9.0,NaN,12.0,NaN


Since the 'c' and 'e' columns are not found in both DataFrame objects, they appear
as all missing in the result. The same holds for the rows whose labels are not common
to both objects.

If we add DataFrame objects with no column or row labels in common, the result
will contain all nulls:

In [118]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})

In [119]:
df1

,A
0,1
1,2


In [120]:
df2

,B
0,3
1,4


In [121]:
df1 - df2

,A,B
0,NaN,NaN
1,NaN,NaN


#### Arithmetic methods with fill values

In arithmetic operations between differently indexed objects, you might want to fill
with a special value, like 0, when an axis label is found in one object but not the other:

In [122]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list('abcde'))
df2.loc[1, 'b'] = np.nan

In [123]:
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [124]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


Adding these together results in NA values in the locations that don’t overlap:

In [125]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


Using the add method on df1, I pass df2 and an argument to fill_value:

In [126]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


when reindexing a Series or DataFrame, we can also specify a different fill
value:

In [127]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


#### Operations between DataFrame and Series

As with NumPy arrays of different dimensions, arithmetic between DataFrame and
Series is also defined. First, as a motivating example, consider the difference between
a two-dimensional array and one of its rows

In [128]:
arr = np.arange(12.).reshape((3, 4))
arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [129]:
arr[0]

array([0., 1., 2., 3.])

In [130]:
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

When we subtract arr[0] from arr, the subtraction is performed once for each row.
This is referred to as *broadcasting*:

In [131]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Aveiro', 'Coimbra', 'Lisboa', 'Porto'])
series = frame.iloc[0]

In [132]:
frame

,b,d,e
Aveiro,0.0,1.0,2.0
Coimbra,3.0,4.0,5.0
Lisboa,6.0,7.0,8.0
Porto,9.0,10.0,11.0


In [133]:
series = frame.iloc[0]
series

b    0.0
d    1.0
e    2.0
Name: Aveiro, dtype: float64

By default, arithmetic between DataFrame and Series matches the index of the Series
on the DataFrame’s columns, broadcasting down the rows:

In [134]:
frame - series

,b,d,e
Aveiro,0.0,0.0,0.0
Coimbra,3.0,3.0,3.0
Lisboa,6.0,6.0,6.0
Porto,9.0,9.0,9.0


If an index value is not found in either the DataFrame’s columns or the Series’s index,
the objects will be reindexed to form the union:

In [135]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2

,b,d,e,f
Aveiro,0.0,NaN,3.0,NaN
Coimbra,3.0,NaN,6.0,NaN
Lisboa,6.0,NaN,9.0,NaN
Porto,9.0,NaN,12.0,NaN


If you want to instead broadcast over the columns, matching on the rows, you have to
use one of the arithmetic methods.

In [136]:
series3 = frame['d']
frame

,b,d,e
Aveiro,0.0,1.0,2.0
Coimbra,3.0,4.0,5.0
Lisboa,6.0,7.0,8.0
Porto,9.0,10.0,11.0


In [137]:
series3 = frame['d']
series3

Aveiro      1.0
Coimbra     4.0
Lisboa      7.0
Porto      10.0
Name: d, dtype: float64

In [138]:
frame.sub(series3, axis='index')

,b,d,e
Aveiro,-1.0,0.0,1.0
Coimbra,-1.0,0.0,1.0
Lisboa,-1.0,0.0,1.0
Porto,-1.0,0.0,1.0


### Function Application and Mapping

NumPy ufuncs (element-wise array methods) also work with pandas objects:

In [139]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Coimbra', 'Porto', 'Aveiro', 'Lisboa'])
frame

,b,d,e
Coimbra,-0.204708,0.478943,-0.519439
Porto,-0.555730,1.965781,1.393406
Aveiro,0.092908,0.281746,0.769023
Lisboa,1.246435,1.007189,-1.296221


In [140]:
np.abs(frame)

,b,d,e
Coimbra,0.204708,0.478943,0.519439
Porto,0.555730,1.965781,1.393406
Aveiro,0.092908,0.281746,0.769023
Lisboa,1.246435,1.007189,1.296221


Another frequent operation is applying a function on one-dimensional arrays to each
column or row. DataFrame’s apply method does exactly this:

In [141]:
f = lambda x: x.max() - x.min()
frame.apply(f)

b    1.802165
d    1.684034
e    2.689627
dtype: float64

Here the function f, which computes the difference between the maximum and minimum
of a Series, is invoked once on each column in frame. The result is a Series having
the columns of frame as its index.

If you pass axis='columns' to apply, the function will be invoked once per row
instead:

In [142]:
frame.apply(f, axis='columns')

Coimbra    0.998382
Porto      2.521511
Aveiro     0.676115
Lisboa     2.542656
dtype: float64

Many of the most common array statistics (like sum and mean) are DataFrame methods,
so using apply is not necessary.

The function passed to apply need not return a scalar value; it can also return a Series
with multiple values:

In [143]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

,b,d,e
min,-0.555730,0.281746,-1.296221
max,1.246435,1.965781,1.393406


Element-wise Python functions can be used, too. Suppose you wanted to compute a
formatted string from each floating-point value in frame. You can do this with apply
map:

In [144]:
format = lambda x: '%.2f' % x
frame.applymap(format)

,b,d,e
Coimbra,-0.20,0.48,-0.52
Porto,-0.56,1.97,1.39
Aveiro,0.09,0.28,0.77
Lisboa,1.25,1.01,-1.30


The reason for the name applymap is that Series has a map method for applying an
element-wise function:

In [145]:
frame['e'].map(format)

Coimbra    -0.52
Porto       1.39
Aveiro      0.77
Lisboa     -1.30
Name: e, dtype: object

### Sorting and Ranking

Sorting a dataset by some criterion is another important built-in operation. To sort
lexicographically by row or column index, use the sort_index method, which returns
a new, sorted object:

In [146]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

With a DataFrame, you can sort by index on either axis:

In [147]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])

In [148]:
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [149]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [150]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


The data is sorted in ascending order by default, but can be sorted in descending
order, too:

In [151]:
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


To sort a Series by its values, use its sort_values method:

In [152]:
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

Any missing values are sorted to the end of the Series by default:

In [153]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

When sorting a DataFrame, you can use the data in one or more columns as the sort
keys. To do so, pass one or more column names to the by option of sort_values:

In [154]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [155]:
frame.sort_values(by='b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


To sort by multiple columns, pass a list of names:

In [156]:
frame.sort_values(by=['a', 'b'])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


Ranking assigns ranks from one through the number of valid data points in an array.
The rank methods for Series and DataFrame are the place to look; by default rank
breaks ties by assigning each group the mean rank:

In [157]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

Ranks can also be assigned according to the order in which they’re observed in the
data:.

In [158]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

Here, instead of using the average rank 6.5 for the entries 0 and 2, they instead have
been set to 6 and 7 because label 0 precedes label 2 in the data.

You can rank in descending order, too:

In [159]:
# Assign tie values the maximum rank in the group
obj.rank(ascending=False, method='max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

DataFrame can compute ranks over the rows or the columns:

In [160]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1],
                      'c': [-2, 5, 8, -2.5]})
frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [161]:
frame.rank(axis='columns')

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


### Axis Indexes with Duplicate Labels

While many pandas functions (like reindex) require that the labels be
unique, it’s not mandatory. Let’s consider a small Series with duplicate indices:

In [162]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

The index’s is_unique property can tell you whether its labels are unique or not:

In [163]:
obj.index.is_unique

False

Data selection is one of the main things that behaves differently with duplicates.
Indexing a label with multiple entries returns a Series, while single entries return a
scalar value

In [164]:
obj['a']

a    0
a    1
dtype: int64

In [165]:
obj['c']

4

This can make your code more complicated, as the output type from indexing can
vary based on whether a label is repeated or not.

The same logic extends to indexing rows in a DataFrame:

In [166]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

,0,1,2
a,0.274992,0.228913,1.352917
a,0.886429,-2.001637,-0.371843
b,1.669025,-0.438570,-0.539741
b,0.476985,3.248944,-1.021228


In [167]:
df.loc['b']

,0,1,2
b,1.669025,-0.438570,-0.539741
b,0.476985,3.248944,-1.021228


## Summarizing and Computing Descriptive Statistics

pandas objects are equipped with a set of common mathematical and statistical methods.
Most of these fall into the category of reductions or summary statistics, methods
that extract a single value (like the sum or mean) from a Series or a Series of values
from the rows or columns of a DataFrame. Compared with the similar methods
found on NumPy arrays, they have built-in handling for missing data. Consider

In [168]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


Calling DataFrame’s sum method returns a Series containing column sums:

In [169]:
df.sum()

one    9.25
two   -5.80
dtype: float64

Passing axis='columns' or axis=1 sums across the columns instead:

In [170]:
df.sum(axis='columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

NA values are excluded unless the entire slice (row or column in this case) is NA.
This can be disabled with the skipna option:

In [171]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

Some methods, like idxmin and idxmax, return indirect statistics like the index value
where the minimum or maximum values are attained:

In [172]:
df.idxmax()

one    b
two    d
dtype: object

Other methods are accumulations:

In [173]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


Another type of method is neither a reduction nor an accumulation. describe is one
such example, producing multiple summary statistics in one shot:

In [174]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


On non-numeric data, describe produces alternative summary statistics:

In [175]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

### Correlation and Covariance

Some summary statistics, like correlation and covariance, are computed from pairs of
arguments. Let’s consider some DataFrames of stock prices and volumes obtained
from Yahoo! Finance using the add-on pandas-datareader package. If you don’t
have it installed already, it can be obtained via conda or pip:

In [177]:
conda install pandas-datareader

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
- DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/osx-64/current_repodata.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/osx-64/current_repodata.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 200 None
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.2
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages upd

In [176]:
price = pd.read_pickle('examples/yahoo_price.pkl')
volume = pd.read_pickle('examples/yahoo_volume.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'examples/yahoo_price.pkl'

```python
import pandas_datareader.data as web
all_data = {ticker: web.get_data_yahoo(ticker)
            for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}

price = pd.DataFrame({ticker: data['Adj Close']
                     for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume']
                      for ticker, data in all_data.items()})
```

We now compute percent changes of the prices:

In [178]:
returns = price.pct_change()
returns.tail()

NameError: name 'price' is not defined

The corr method of Series computes the correlation of the overlapping, non-NA,
aligned-by-index values in two Series. Relatedly, cov computes the covariance:

In [ ]:
returns['MSFT'].corr(returns['IBM'])

In [ ]:
returns['MSFT'].cov(returns['IBM'])

DataFrame’s corr and cov methods, on the other hand, return a full correlation or
covariance matrix as a DataFrame, respectively:

In [ ]:
returns.corr()

In [ ]:
returns.cov()

Using DataFrame’s corrwith method, you can compute pairwise correlations
between a DataFrame’s columns or rows with another Series or DataFrame. Passing a
Series returns a Series with the correlation value computed for each column:

In [ ]:
returns.corrwith(returns.IBM)

Passing a DataFrame computes the correlations of matching column names. Here I
compute correlations of percent changes with volume:

In [ ]:
returns.corrwith(volume)

Passing axis='columns' does things row-by-row instead. In all cases, the data points
are aligned by label before the correlation is computed.

### Unique Values, Value Counts, and Membership

Another class of related methods extracts information about the values contained in a
one-dimensional Series.

In [179]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

The first function is unique, which gives you an array of the unique values in a Series:

In [180]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

The unique values are not necessarily returned in sorted order, but could be sorted
after the fact if needed (uniques.sort()). Relatedly, value_counts computes a Series
containing value frequencies:

In [181]:
obj.value_counts()

c    3
a    3
b    2
d    1
dtype: int64

The Series is sorted by value in descending order as a convenience. value_counts is
also available as a top-level pandas method that can be used with any array or
sequence:

In [182]:
pd.value_counts(obj.values, sort=False)

c    3
a    3
d    1
b    2
dtype: int64

isin performs a vectorized set membership check and can be useful in filtering a
dataset down to a subset of values in a Series or column in a DataFrame:

In [183]:
obj
mask = obj.isin(['b', 'c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [184]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

Related to isin is the Index.get_indexer method, which gives you an index array
from an array of possibly non-distinct values into another array of distinct values:

In [185]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])
pd.Index(unique_vals).get_indexer(to_match)

array([0, 2, 1, 1, 0, 2])

In some cases, we may want to compute a histogram on multiple related columns in
a DataFrame.

In [186]:
data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                     'Qu2': [2, 3, 1, 2, 3],
                     'Qu3': [1, 5, 2, 4, 4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


In [190]:
data.value_counts

<bound method DataFrame.value_counts of    Qu1  Qu2  Qu3
0    1    2    1
1    3    3    5
2    4    1    2
3    3    2    4
4    4    3    4>

In [189]:
result = data.apply(pd.value_counts).fillna(0)
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0
